### Dados de ondas do modelo Wavewatch III

Dado mensal amostrado de 6 em 6 horas, as variáveis são tempo, direção de onda e altura significativa da onda.

- valores de máximo e mínimo da altura significativa
- média da altura de onda
- série temporal das alturas significativas
- gráfico polar mostrando direção de onda e altura significativa

In [ ]:
import pandas as pd


df = pd.read_excel("./data/2005.02_onda.xlsx").head()

df.head()

Podemos fazer melhor que isso!

In [ ]:
df = pd.read_excel("./data/2005.02_onda.xlsx", parse_cols="A:C").head()

df.head()

Como converter o conjunto de números da coluna tempo para objetos `datetime`?

1) O que temos na coluna tempo?

In [ ]:
date = df['tempo'].ix[0]
date

In [ ]:
date.split()

2) A entrada para objetos `datetime` é `int`

In [ ]:
new_date = []

for d in date.split():
    new_date.append(int(d))

new_date

Podemos converter para `int` mais facilmente assim

In [ ]:
map(int, date.split())

3) Juntando tudo

In [ ]:
from datetime import datetime

datetime(*map(int, date.split()))

4) Criando uma função que recebe o `string` de datas e retorna o objeto `datetime`

In [ ]:
def convert(date):
    return datetime(*map(int, date.split()))

convert(date)

Pronto!  Agora podemos passar essa função para o `pandas.read_excel` e os valores serão convertidos automagicamente

In [ ]:
df = pd.read_excel("./data/2005.02_onda.xlsx",
                   parse_cols="A:C",
                   converters={'tempo': convert})

df.head()

O último passo e usar essa nova coluna com o index

In [ ]:
df.set_index('tempo', inplace=True)

df.tail()

- valores de máximo e minimo da altura significativa e,
- média da altura de onda

In [ ]:
df.describe()

- série temporal das alturas significativas e,
- gráfico polar mostrando direção de onda e altura significativa

In [ ]:
%matplotlib inline

import matplotlib
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(11, 3.25))
df.plot(ax=ax, secondary_y=['Hsig']);

In [ ]:
from oceans import stick_plot
from oceans.ff_tools.ocfis import pol2cart

x, y = pol2cart(df['dir'], df['Hsig'], units='deg')

fig, ax = plt.subplots(figsize=(11, 3.25))

q = stick_plot(df.index.to_pydatetime(), x.values, y.values, ax=ax)

ref = 1
qk = plt.quiverkey(q, 0.1, 0.85, ref,
                  "%s m" % ref,
                  labelpos='N', coordinates='axes')

In [ ]:
import numpy as np

r = np.deg2rad(df['dir'])

fig, ax = plt.subplots(subplot_kw=dict(polar=True))

ax.plot(r, df['Hsig']);

In [ ]:
from windrose import WindroseAxes
from matplotlib import pyplot as plt
import matplotlib.cm as cm


ax = WindroseAxes.from_ax()
ax.contourf(df['dir'], df['Hsig'], blowto=True, cmap=cm.RdBu_r)
ax.set_legend()

In [ ]:
ax = WindroseAxes.from_ax()
ax.box(df['dir'], df['Hsig'], bins=np.arange(0, 1.8, 0.15))
ax.set_legend()

In [ ]:
plt.hist(df['dir'], bins=15);

In [ ]:
plt.hist(df['Hsig'], bins=15);

Análise de valores extremos

In [ ]:
import wafo.stats as ws


wei = ws.weibull_min.fit2(df['Hsig'])

fig, ax = plt.subplots(figsize=(7, 7))
ws.probplot(df['Hsig'], wei.par, dist='weibull_min', plot=ax);

In [ ]:
'{} < {} < {}'.format(wei.par_lower[0], wei.par[0], wei.par_upper[0])

In [ ]:
gev = ws.genextreme.fit2(df['Hsig'])

fig, ax = plt.subplots(figsize=(9, 9))
gev.plotesf()

Generalized Extreme Value distribution

In [ ]:
import wafo.kdetools as wk

fig, ax = plt.subplots(figsize=(9, 9))

wk.TKDE(df['Hsig'], L2=0.5)(output='plot').plot('c--')
gev.plotepdf(symb1='g-')
wei.plotepdf(symb1='r-')

In [ ]:
from scipy import stats
import matplotlib.pyplot as plt

def make_plot():
    fig, ax = plt.subplots(figsize=(9, 9))
    ax.hist(df['Hsig'], bins=10, normed=True, alpha=0.5);

    x = np.linspace(df['Hsig'].min(), df['Hsig'].max(), 1000)

    # Exponential Weibull
    params = stats.exponweib.fit(df['Hsig'])
    weib = stats.exponweib.pdf(x, *params)
    ax.plot(x, weib, label='exponweib')

    # Weibull minimum (Frechet right)
    params = stats.weibull_min.fit(df['Hsig'])
    weib = stats.weibull_min.pdf(x, *params)
    ax.plot(x, weib, label='weibull_min')

    # By hand!
    def weib(x, c):
        return c*x**(c-1)*np.exp(-x**c)
    c, loc, scale = params
    y = (x-loc) / scale
    w = weib(y, c) / scale
    ax.plot(x, w, '--', label='Manual')

    leg = ax.legend()

In [ ]:
make_plot()

Weibull

$$ c x^{c-1} e^{-x^c} $$

In [ ]:
c, loc, scale = stats.weibull_min.fit(df['Hsig'])

c, loc, scale

In [ ]:
df['Hsig'].describe()[['max', '75%', 'mean', 'min', 'std']]

Classe para "wave calculations."

In [ ]:
from oceans import Waves

w = Waves(h=100, T=15, L=None)

w.C, w.Cg, w.omega, w.k